In [ ]:
# Import standard modules.

# Import supplemental modules.
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Notes on notation

# All citations are from:

# Tsyganenko et al, Ann. Geophys., 33, 1–11, 2015
# doi:10.5194/angeo-33-1-2015

# PpCcLl = page, column, line

# Notes on variable names:
# rho = Radius in cylindrical solar magetic coordinates (R_E)
# phi = Longitude in cylindrical solar magetic coordinates (radians)
# psi = Tilt angle of Earth dipole relative to SW flow (radians)
# P = Solar wind dynamic (ram) pressure, units of nP
# By = y-component of solar wind, units of nT
# Bz = z-component of solar wind, units of nT

In [ ]:
# Define the parameters and equations of the model.

# Define the fitted values and uncertainties (Table 1)
# Uncertainties are RMS absolute deviations.
RH0, RH0_rms = 11.02, 0.05          
RH1, RH1_rms = 6.05, 0.88
RH2, RH2_rms = 0.84, 0.09
RH3, RH3_rms = -2.28, 0.08
RH4, RH4_rms = -0.25, 0.37
RH5, RH5_rms = -0.96, 0.16
T0, T0_rms = 0.29, 0.02
T1, T1_rms = 0.18, 0.08
a00, a00_rms = 2.91, 0.02
a01, a01_rms = -0.16, 0.07
a02, a02_rms = 0.56, 0.03
a10, a10_rms = 1.89, 0.03
a11, a11_rms = 0.06, 0.04
a12, a12_rms = 0.49, 0.04
alpha0, alpha0_rms = 7.13, 0.06
alpha1, alpha1_rms = 4.87, 0.07
alpha2, alpha2_rms = -0.22, 0.12
alpha3, alpha3_rms = -0.14, 0.04
chi, chi_rms = -0.29, 0.03  # 
beta0, beta0_rms = 2.18, 0.09
beta1, beta1_rms = 0.40, 0.11

# Constants for input scaling

# Radius scale (R_E)
rho0 = 10.0  # P5C2L7

# Pressure scale (nPa)
Pmean = 2.0  # P6C1L3

# Magnetic field scale (nT)
By0 = 5.0  # P5C2L6
Bz0 = 5.0  # P6C1L3

# IMPORTANT NOTE: For numpy reasons, all inputs to the functions below
# must have the same shape.

# Define the scaling functions for solar wind data.
def fP_empirical(P):
    """Equation 10"""
    return (P/Pmean)**chi - 1

def fBz_empirical(Bz):
    """Equation 10"""
    return Bz/Bz0

# Define the empirical equations for the model parameters as a function of
# solar wind variables and coordinates.

# Model is in a cylindrical solar magnetic system centered on Earth.
# rho = Geocentric distance (R_E)
# phi = Longitude (radians) - WHERE IS phi = 0? Relative to +x?
# z = height above the GSM XY plane (R_E)

# fP = scaled pressure (no units)
# fBz = scaled Bz (no units)

def RH_empirical(fP, fBz, phi):
    """Equation 4"""
    return RH0 + RH1*fP + RH2*fBz + (RH3 + RH4*fP + RH5*fBz)*np.cos(phi)

def T_empirical(fP, fBz, phi):
    """Equation 5"""
    return T0 + T1*fP

def a0_empirical(fP, fBz, phi):
    """Equation 6"""
    return a00 + a01*fP + a02*fBz

def a1_empirical(fP, fBz, phi):
    """Equation 7"""
    return a10 + a11*fP + a12*fBz

def alpha_empirical(fP, fBz, phi):
    """Equation 8"""
    return alpha0 + alpha1*np.cos(phi) + alpha2*fP + alpha3*fBz

def beta_empirical(fP, fBz, phi):
    """Equation 9"""
    return beta0 + beta1*fBz

# Define the empirical model for the z-coordinate of the current sheet.
def Zs_empirical(P, By, Bz, psi, rho, phi):
    """Equation 3"""
    # Scale the solar wind data.
    fP = fP_empirical(P)
    fBz = fBz_empirical(Bz)

    # Compute the model parameters.
    RH = RH_empirical(fP, fBz, phi)
    T = T_empirical(fP, fBz, phi)
    a0 = a0_empirical(fP, fBz, phi)
    a1 = a1_empirical(fP, fBz, phi)
    alpha = alpha_empirical(fP, fBz, phi)
    beta = beta_empirical(fP, fBz, phi)

    # Compute the current sheet z-coordinate.
    Zs = (
        RH*np.tan(psi) *
        (1 - (1 + (rho/RH)**alpha)**(1/alpha)) *
        (a0 + a1*np.cos(phi)) +
        T*By/By0*(rho/rho0)**beta*np.sin(phi)
    )
    return Zs

In [ ]:
# Plot the scaling functions for the input variables.

# Note that chi < 0, so fP decreases with increasing P.
# Figure parameters
figsize = (10, 5)
NROWS = 1
NCOLS = 2
MEAN_VALUE_COLOR = 'red'

# Set up figure for side-by-side plots.
fig = plt.figure(figsize=figsize)
fig.suptitle('Scaled input variables')
gs = mpl.gridspec.GridSpec(NROWS, NCOLS)

# Left plot: Pressure scaling
Pmin, Pmax, nP = 0, 10, 101
P = np.linspace(Pmin, Pmax, nP)[1:]  # Skip P = 0
fPmin, fPmax = -1.0, 3.0
fP = fP_empirical(P)
ax = fig.add_subplot(gs[0])
ax.grid()
ax.set_xlabel("$P_{dyn}$ (nPa)")
ax.set_ylabel("$f_P$")
ax.set_title("Scaled solar wind dynamic pressure ($f_P$)")
ax.plot(P, fP)
# Plot vertical line at scale value.
ax.vlines(Pmean, fPmin, fPmax, color=MEAN_VALUE_COLOR)

# Right plot: Bz scaling
Bzmin, Bzmax, nBz = -2*Bz0, 2*Bz0, 101
Bz = np.linspace(Bzmin, Bzmax, nBz)
fBzmin, fBzmax = -2.0, 2.0
fBz = fBz_empirical(Bz)
ax = fig.add_subplot(gs[1])
ax.grid()
ax.set_xlabel("$B_z$ (nT)")
ax.set_ylabel("$f_{Bz}$")
ax.set_title("Scaled solar wind $B_z$ ($f_{Bz}$)")
ax.plot(Bz, fBz)
# Plot vertical line at scale value.
ax.vlines(Bz0, fBzmin, fBzmax, color=MEAN_VALUE_COLOR)
plt.show()

In [ ]:
# Plot the empirical equation for RH.

# Define ranges for scaled pressure and Bz.
fPmin, fPmax, nfP = -1.0, 2.0, 101
fP = np.linspace(fPmin, fPmax, nfP)
fBzmin, fBzmax, nfBz = -2.0, 2.0, 101
fBz = np.linspace(fBzmin, fBzmax, nfBz)

# Make a meshgrid for evaluation.
FP, FBZ = np.meshgrid(fP, fBz)
# Both are shape (nfBz, nfP).
# The nfBz rows of FP are the nfP values of fP.
# The nfP columns of FBZ are the nfBz values of fBz.

# Make PHI have the same shape.
PHI = np.full(FP.shape, 0.0)

# Compute the value of RH at each point.
RH = RH_empirical(FP, FBZ, PHI)
# Shape is (nfBz, nfP)

# Plot as image.
fig = plt.figure()
ax = fig.add_subplot()
aspect = nfP/nfBz
im = ax.imshow(RH, origin='lower', aspect=aspect)
fig.colorbar(im)

# Decorate the plot.
ax.grid()
ax.set_xlabel("$f_P$")
ax.set_ylabel("$f_{Bz}$")
ax.set_title("$R_H$ (empirical)")

# Compute and show ticks and labels.
# Tick positions are in pixel coordinates.
N_X_TICKS = 5
x_tick_pos = np.linspace(0, nfP - 1, N_X_TICKS)
x_tick_labels = ["%.1f" % (fPmin + x/(nfP - 1)*(fPmax - fPmin)) for x in x_tick_pos]
ax.set_xticks(x_tick_pos)
ax.set_xticklabels(x_tick_labels)
N_Y_TICKS = 5
y_tick_pos = np.linspace(0, nfBz - 1, N_Y_TICKS)
y_tick_labels = ["%.1f" % (fBzmin + y/(nfBz - 1)*(fBzmax - fBzmin)) for y in y_tick_pos]
ax.set_yticks(y_tick_pos)
ax.set_yticklabels(y_tick_labels)
plt.show()

In [ ]:
# Verify the results are properly oriented.
print(FP.shape, FBZ.shape, PHI.shape, RH.shape)
i, j = 0, 0  # Lower left
print(FP[i, j], FBZ[i, j], PHI[i, j], RH[i, j], RH_empirical(FP[i, j], FBZ[i, j], PHI[i, j]))
i, j = 0, -1  # Lower right
print(FP[i, j], FBZ[i, j], PHI[i, j], RH[i, j], RH_empirical(FP[i, j], FBZ[i, j], PHI[i, j]))
i, j = -1, -1  # Upper right
print(FP[i, j], FBZ[i, j], PHI[i, j], RH[i, j], RH_empirical(FP[i, j], FBZ[i, j], PHI[i, j]))
i, j = -1, 0  # Upper left
print(FP[i, j], FBZ[i, j], PHI[i, j], RH[i, j], RH_empirical(FP[i, j], FBZ[i, j], PHI[i, j]))

In [ ]:
# Plot the empirical equation for T.

# Define range for scaled pressure.
fPmin, fPmax, nfP = -1.0, 2.0, 101
fP = np.linspace(fPmin, fPmax, nfP)
fBz = None
phi = None

# Create the figure.
fig = plt.figure()

# Compute the parameter at each point, then reorient.
T = T_empirical(fP, fBz, phi)

# Create the plot.
ax = fig.add_subplot()
ax.grid()
ax.plot(fP, T)
ax.set_xlabel("$f_P$")
ax.set_ylabel("$T$")
ax.set_title("$T$ (empirical)")
plt.show()

In [ ]:
# Plot the empirical equation for a0.

# Define ranges for scaled pressure and Bz.
fPmin, fPmax, nfP = -1.0, 2.0, 101
fP = np.linspace(fPmin, fPmax, nfP)
fBzmin, fBzmax, nfBz = -2.0, 2.0, 101
fBz = np.linspace(fBzmin, fBzmax, nfBz)

# Make a meshgrid for evaluation.
FP, FBZ = np.meshgrid(fP, fBz)
# Both are shape (nfBz, nfP).
# The nfBz rows of FP are the nfP values of fP.
# The nfP columns of FBZ are the nfBz values of fBz.

# Make PHI have the same shape.
PHI = np.full(FP.shape, 0.0)

# Compute the value of a0 at each point.
A0 = a0_empirical(FP, FBZ, PHI)
# Shape is (nfBz, nfP)

# Plot as image.
fig = plt.figure()
ax = fig.add_subplot()
aspect = nfP/nfBz
im = ax.imshow(A0, origin='lower', aspect=aspect)
fig.colorbar(im)

# Decorate the plot.
ax.grid()
ax.set_xlabel("$f_P$")
ax.set_ylabel("$f_{Bz}$")
ax.set_title("$a_0$ (empirical)")

# Compute and show ticks and labels.
# Tick positions are in pixel coordinates.
N_X_TICKS = 5
x_tick_pos = np.linspace(0, nfP - 1, N_X_TICKS)
x_tick_labels = ["%.1f" % (fPmin + x/(nfP - 1)*(fPmax - fPmin)) for x in x_tick_pos]
ax.set_xticks(x_tick_pos)
ax.set_xticklabels(x_tick_labels)
N_Y_TICKS = 5
y_tick_pos = np.linspace(0, nfBz - 1, N_Y_TICKS)
y_tick_labels = ["%.1f" % (fBzmin + y/(nfBz - 1)*(fBzmax - fBzmin)) for y in y_tick_pos]
ax.set_yticks(y_tick_pos)
ax.set_yticklabels(y_tick_labels)
plt.show()

In [ ]:
# Verify the results are properly oriented.
print(FP.shape, FBZ.shape, PHI.shape, A0.shape)
i, j = 0, 0  # Lower left
print(FP[i, j], FBZ[i, j], PHI[i, j], A0[i, j], a0_empirical(FP[i, j], FBZ[i, j], PHI[i, j]))
i, j = 0, -1  # Lower right
print(FP[i, j], FBZ[i, j], PHI[i, j], A0[i, j], a0_empirical(FP[i, j], FBZ[i, j], PHI[i, j]))
i, j = -1, -1  # Upper right
print(FP[i, j], FBZ[i, j], PHI[i, j], A0[i, j], a0_empirical(FP[i, j], FBZ[i, j], PHI[i, j]))
i, j = -1, 0  # Upper left
print(FP[i, j], FBZ[i, j], PHI[i, j], A0[i, j], a0_empirical(FP[i, j], FBZ[i, j], PHI[i, j]))

In [ ]:
# Plot the empirical equation for a1.

# Define ranges for scaled pressure and Bz.
fPmin, fPmax, nfP = -1.0, 2.0, 101
fP = np.linspace(fPmin, fPmax, nfP)
fBzmin, fBzmax, nfBz = -2.0, 2.0, 101
fBz = np.linspace(fBzmin, fBzmax, nfBz)

# Make a meshgrid for evaluation.
FP, FBZ = np.meshgrid(fP, fBz)
# Both are shape (nfBz, nfP).
# The nfBz rows of FP are the nfP values of fP.
# The nfP columns of FBZ are the nfBz values of fBz.

# Make PHI have the same shape.
PHI = np.full(FP.shape, 0.0)

# Compute the value of a1 at each point.
A1 = a1_empirical(FP, FBZ, PHI)
# Shape is (nfBz, nfP)

# Plot as image.
fig = plt.figure()
ax = fig.add_subplot()
aspect = nfP/nfBz
im = ax.imshow(A1, origin='lower', aspect=aspect)
fig.colorbar(im)

# Decorate the plot.
ax.grid()
ax.set_xlabel("$f_P$")
ax.set_ylabel("$f_{Bz}$")
ax.set_title("$a_1$ (empirical)")

# Compute and show ticks and labels.
# Tick positions are in pixel coordinates.
N_X_TICKS = 5
x_tick_pos = np.linspace(0, nfP - 1, N_X_TICKS)
x_tick_labels = ["%.1f" % (fPmin + x/(nfP - 1)*(fPmax - fPmin)) for x in x_tick_pos]
ax.set_xticks(x_tick_pos)
ax.set_xticklabels(x_tick_labels)
N_Y_TICKS = 5
y_tick_pos = np.linspace(0, nfBz - 1, N_Y_TICKS)
y_tick_labels = ["%.1f" % (fBzmin + y/(nfBz - 1)*(fBzmax - fBzmin)) for y in y_tick_pos]
ax.set_yticks(y_tick_pos)
ax.set_yticklabels(y_tick_labels)
plt.show()

In [ ]:
# Verify the results are properly oriented.
print(FP.shape, FBZ.shape, PHI.shape, A1.shape)
i, j = 0, 0  # Lower left
print(FP[i, j], FBZ[i, j], PHI[i, j], A1[i, j], a1_empirical(FP[i, j], FBZ[i, j], PHI[i, j]))
i, j = 0, -1  # Lower right
print(FP[i, j], FBZ[i, j], PHI[i, j], A1[i, j], a1_empirical(FP[i, j], FBZ[i, j], PHI[i, j]))
i, j = -1, -1  # Upper right
print(FP[i, j], FBZ[i, j], PHI[i, j], A1[i, j], a1_empirical(FP[i, j], FBZ[i, j], PHI[i, j]))
i, j = -1, 0  # Upper left
print(FP[i, j], FBZ[i, j], PHI[i, j], A1[i, j], a1_empirical(FP[i, j], FBZ[i, j], PHI[i, j]))

In [ ]:
# Plot the empirical equation for alpha.

# Define ranges for scaled pressure and Bz.
fPmin, fPmax, nfP = -1.0, 2.0, 101
fP = np.linspace(fPmin, fPmax, nfP)
fBzmin, fBzmax, nfBz = -2.0, 2.0, 101
fBz = np.linspace(fBzmin, fBzmax, nfBz)

# Make a meshgrid for evaluation.
FP, FBZ = np.meshgrid(fP, fBz)
# Both are shape (nfBz, nfP).
# The nfBz rows of FP are the nfP values of fP.
# The nfP columns of FBZ are the nfBz values of fBz.

# Make PHI have the same shape.
PHI = np.full(FP.shape, 0.0)

# Compute the value of alpha at each point.
ALPHA = alpha_empirical(FP, FBZ, PHI)
# Shape is (nfBz, nfP)

# Plot as image.
fig = plt.figure()
ax = fig.add_subplot()
aspect = nfP/nfBz
im = ax.imshow(ALPHA, origin='lower', aspect=aspect)
fig.colorbar(im)

# Decorate the plot.
ax.grid()
ax.set_xlabel("$f_P$")
ax.set_ylabel("$f_{Bz}$")
ax.set_title(r"$\alpha$ (empirical)")

# Compute and show ticks and labels.
# Tick positions are in pixel coordinates.
N_X_TICKS = 5
x_tick_pos = np.linspace(0, nfP - 1, N_X_TICKS)
x_tick_labels = ["%.1f" % (fPmin + x/(nfP - 1)*(fPmax - fPmin)) for x in x_tick_pos]
ax.set_xticks(x_tick_pos)
ax.set_xticklabels(x_tick_labels)
N_Y_TICKS = 5
y_tick_pos = np.linspace(0, nfBz - 1, N_Y_TICKS)
y_tick_labels = ["%.1f" % (fBzmin + y/(nfBz - 1)*(fBzmax - fBzmin)) for y in y_tick_pos]
ax.set_yticks(y_tick_pos)
ax.set_yticklabels(y_tick_labels)
plt.show()

In [ ]:
# Verify the results are properly oriented.
print(FP.shape, FBZ.shape, PHI.shape, ALPHA.shape)
i, j = 0, 0  # Lower left
print(FP[i, j], FBZ[i, j], PHI[i, j], ALPHA[i, j], alpha_empirical(FP[i, j], FBZ[i, j], PHI[i, j]))
i, j = 0, -1  # Lower right
print(FP[i, j], FBZ[i, j], PHI[i, j], ALPHA[i, j], alpha_empirical(FP[i, j], FBZ[i, j], PHI[i, j]))
i, j = -1, -1  # Upper right
print(FP[i, j], FBZ[i, j], PHI[i, j], ALPHA[i, j], alpha_empirical(FP[i, j], FBZ[i, j], PHI[i, j]))
i, j = -1, 0  # Upper left
print(FP[i, j], FBZ[i, j], PHI[i, j], ALPHA[i, j], alpha_empirical(FP[i, j], FBZ[i, j], PHI[i, j]))

In [ ]:
# Plot the empirical equation for beta.

# Define ranges for scaled pressure and Bz.
fPmin, fPmax, nfP = -1.0, 2.0, 101
fP = np.linspace(fPmin, fPmax, nfP)
fBzmin, fBzmax, nfBz = -2.0, 2.0, 101
fBz = np.linspace(fBzmin, fBzmax, nfBz)

# Make a meshgrid for evaluation.
FP, FBZ = np.meshgrid(fP, fBz)
# Both are shape (nfBz, nfP).
# The nfBz rows of FP are the nfP values of fP.
# The nfP columns of FBZ are the nfBz values of fBz.

# Make PHI have the same shape.
PHI = np.full(FP.shape, 0.0)

# Compute the value of beta at each point.
BETA = beta_empirical(FP, FBZ, PHI)
# Shape is (nfBz, nfP)

# Plot as image.
fig = plt.figure()
ax = fig.add_subplot()
aspect = nfP/nfBz
im = ax.imshow(BETA, origin='lower', aspect=aspect)
fig.colorbar(im)

# Decorate the plot.
ax.grid()
ax.set_xlabel("$f_P$")
ax.set_ylabel("$f_{Bz}$")
ax.set_title(r"$\beta$ (empirical)")

# Compute and show ticks and labels.
# Tick positions are in pixel coordinates.
N_X_TICKS = 5
x_tick_pos = np.linspace(0, nfP - 1, N_X_TICKS)
x_tick_labels = ["%.1f" % (fPmin + x/(nfP - 1)*(fPmax - fPmin)) for x in x_tick_pos]
ax.set_xticks(x_tick_pos)
ax.set_xticklabels(x_tick_labels)
N_Y_TICKS = 5
y_tick_pos = np.linspace(0, nfBz - 1, N_Y_TICKS)
y_tick_labels = ["%.1f" % (fBzmin + y/(nfBz - 1)*(fBzmax - fBzmin)) for y in y_tick_pos]
ax.set_yticks(y_tick_pos)
ax.set_yticklabels(y_tick_labels)
plt.show()

In [ ]:
# Verify the results are properly oriented.
print(FP.shape, FBZ.shape, PHI.shape, BETA.shape)
i, j = 0, 0  # Lower left
print(FP[i, j], FBZ[i, j], PHI[i, j], BETA[i, j], beta_empirical(FP[i, j], FBZ[i, j], PHI[i, j]))
i, j = 0, -1  # Lower right
print(FP[i, j], FBZ[i, j], PHI[i, j], BETA[i, j], beta_empirical(FP[i, j], FBZ[i, j], PHI[i, j]))
i, j = -1, -1  # Upper right
print(FP[i, j], FBZ[i, j], PHI[i, j], BETA[i, j], beta_empirical(FP[i, j], FBZ[i, j], PHI[i, j]))
i, j = -1, 0  # Upper left
print(FP[i, j], FBZ[i, j], PHI[i, j], BETA[i, j], beta_empirical(FP[i, j], FBZ[i, j], PHI[i, j]))

In [ ]:
# Reconstruct Figure 5
P_dyn = 2.0
By_IMF = 0.0
Bz_IMF = 0.0
psi = np.radians(30)

# Domain limits and point spacing
xmin, xmax, nx = -30.0, 10.0, 401
ymin, ymax, ny = -20.0, 20.0, 401
x = np.linspace(xmin, xmax, nx)
y = np.linspace(ymin, ymax, ny)

# Create the grid of evaluation points.
X, Y = np.meshgrid(x, y)

# Compute the cylindrical equivalents.
RHO = np.sqrt(X**2 + Y**2)
PHI = np.arctan2(Y, X)

# Create arrays for the other arguments.
P = np.full(X.shape, P_dyn)
BY = np.full(X.shape, By_IMF)
BZ = np.full(X.shape, Bz_IMF)
PSI = np.full(X.shape, psi)

# Evaluate the NCS height at each grid point.
# Compare to Figure 5.
ZS = Zs_empirical(P, BY, BZ, PSI, RHO, PHI)

# Create the figure.
figsize = (10, 5)
NROWS = 1
NCOLS = 2
fig = plt.figure(figsize=figsize)
fig.suptitle('Figure 5')
gs = mpl.gridspec.GridSpec(NROWS, NCOLS)

# Left plot: XZ (SM) plane
i = ny//2
XX = X[i]
ZZ = ZS[i]
ax = fig.add_subplot(gs[0])
ax.grid()
ax.set_xlabel("$X_{SM}$")
ax.set_ylabel("$Z_{SM}$")
ax.set_aspect('equal')
plt.plot(XX, ZZ, color='red')
ax.set_xbound(-30.0, 10.0)
ax.set_ybound(-25.0, 25.0)
ax.invert_xaxis()

# Right plot: surface in SM coordinates
ax = fig.add_subplot(gs[1], projection='3d')

# Plot the surface.
ax.plot_wireframe(X, Y, ZS)

# Decorate the plot.
ax.set_xlabel(r'$X_{SM}$')
ax.set_ylabel(r'$Y_{SM}$')
ax.set_zlabel(r'$Z_{SM}$')

# # Set the viewing position to down the +Y axis toward the origin for the
# # SM XZ plane (degrees).
# # Compare to Figure 5.
# # elev, azim, roll = 0.0, 90.0, 0
# elev, azim, roll = 45.0, 45.0, 0
# ax.view_init(elev=elev, azim=azim, roll=roll)

In [ ]:
# Reconstruct Figure 6

# Left plot
By_IMF = 0.0
Bz_IMF = 0.0
psi = np.radians(30)

# Domain limits and point spacing
xmin, xmax, nx = -30.0, 10.0, 401
ymin, ymax, ny = -20.0, 20.0, 401
x = np.linspace(xmin, xmax, nx)
y = np.linspace(ymin, ymax, ny)

# Create the grids of evaluation points and variables.
X, Y = np.meshgrid(x, y)
RHO = np.sqrt(X**2 + Y**2)
PHI = np.arctan2(Y, X)
BY = np.full(X.shape, By_IMF)
BZ = np.full(X.shape, Bz_IMF)
PSI = np.full(X.shape, psi)

# Create the figure.
figsize = (10, 5)
NROWS = 1
NCOLS = 2
fig = plt.figure(figsize=figsize)
fig.suptitle('Figure 6')
gs = mpl.gridspec.GridSpec(NROWS, NCOLS)

# Evaluate the NCS height at each grid point.
P_dyn = 0.5
P = np.full(X.shape, P_dyn)
ZS = Zs_empirical(P, BY, BZ, PSI, RHO, PHI)
i = ny//2
XX = X[i]
ZZ = ZS[i]
ax = fig.add_subplot(gs[0])
ax.grid()
ax.set_xlabel("$X_{SM}$")
ax.set_ylabel("$Z_{SM}$")
ax.set_aspect('equal')
ax.plot(XX, ZZ, color='red')
ax.set_xbound(-30.0, 10.0)
ax.set_ybound(-25.0, 25.0)
ax.invert_xaxis()

# Evaluate the NCS height at each grid point.
P_dyn = 6.0
P = np.full(X.shape, P_dyn)
ZS = Zs_empirical(P, BY, BZ, PSI, RHO, PHI)
i = ny//2
XX = X[i]
ZZ = ZS[i]
ax = fig.add_subplot(gs[1])
ax.grid()
ax.set_xlabel("$X_{SM}$")
ax.set_ylabel("$Z_{SM}$")
ax.set_aspect('equal')
ax.plot(XX, ZZ, color='red')
ax.set_xbound(-30.0, 10.0)
ax.set_ybound(-25.0, 25.0)
ax.invert_xaxis()

plt.show()

In [ ]:
# Reconstruct Figure 7

# Left plot
P_dyn = 2.0
By_IMF = 0.0
psi = np.radians(30)

# Domain limits and point spacing
xmin, xmax, nx = -30.0, 10.0, 401
ymin, ymax, ny = -20.0, 20.0, 401
x = np.linspace(xmin, xmax, nx)
y = np.linspace(ymin, ymax, ny)

# Create the grids of evaluation points and variables.
X, Y = np.meshgrid(x, y)
RHO = np.sqrt(X**2 + Y**2)
PHI = np.arctan2(Y, X)
P = np.full(X.shape, P_dyn)
BY = np.full(X.shape, By_IMF)
PSI = np.full(X.shape, psi)

# Create the figure.
figsize = (10, 5)
NROWS = 1
NCOLS = 2
fig = plt.figure(figsize=figsize)
fig.suptitle('Figure 7')
gs = mpl.gridspec.GridSpec(NROWS, NCOLS)

# Evaluate the NCS height at each grid point.
Bz_IMF = 7.0
BZ = np.full(X.shape, Bz_IMF)
ZS = Zs_empirical(P, BY, BZ, PSI, RHO, PHI)
i = ny//2
XX = X[i]
ZZ = ZS[i]
ax = fig.add_subplot(gs[0])
ax.grid()
ax.set_xlabel("$X_{SM}$")
ax.set_ylabel("$Z_{SM}$")
ax.set_aspect('equal')
ax.plot(XX, ZZ, color='red')
ax.set_xbound(-30.0, 10.0)
ax.set_ybound(-25.0, 25.0)
ax.invert_xaxis()

# Evaluate the NCS height at each grid point.
Bz_IMF = -7.0
BZ = np.full(X.shape, Bz_IMF)
ZS = Zs_empirical(P, BY, BZ, PSI, RHO, PHI)
i = ny//2
XX = X[i]
ZZ = ZS[i]
ax = fig.add_subplot(gs[1])
ax.grid()
ax.set_xlabel("$X_{SM}$")
ax.set_ylabel("$Z_{SM}$")
ax.set_aspect('equal')
ax.plot(XX, ZZ, color='red')
ax.set_xbound(-30.0, 10.0)
ax.set_ybound(-25.0, 25.0)
ax.invert_xaxis()

plt.show()

In [ ]:
# Reconstruct Figure 8

# Left plot
P_dyn = 2.0
By_IMF = 0.0
psi = np.radians(30)

# Domain limits and point spacing
xmin, xmax, nx = -30.0, 10.0, 401
ymin, ymax, ny = -20.0, 20.0, 401
x = np.linspace(xmin, xmax, nx)
y = np.linspace(ymin, ymax, ny)

# Create the grids of evaluation points and variables.
X, Y = np.meshgrid(x, y)
RHO = np.sqrt(X**2 + Y**2)
PHI = np.arctan2(Y, X)
P = np.full(X.shape, P_dyn)
BY = np.full(X.shape, By_IMF)
PSI = np.full(X.shape, psi)

# Create the figure.
figsize = (10, 5)
NROWS = 1
NCOLS = 2
fig = plt.figure(figsize=figsize)
fig.suptitle('Figure 8')
gs = mpl.gridspec.GridSpec(NROWS, NCOLS)

# Evaluate the NCS height at each grid point.
Bz_IMF = 7.0
BZ = np.full(X.shape, Bz_IMF)
ZS = Zs_empirical(P, BY, BZ, PSI, RHO, PHI)
i = nx//2
YY = Y[..., i]
ZZ = ZS[..., i]
ax = fig.add_subplot(gs[0])
ax.grid()
ax.set_xlabel("$Y_{SM}$")
ax.set_ylabel("$Z_{SM}$")
ax.set_aspect('equal')
ax.plot(YY, ZZ, color='red')
ax.set_xbound(-25.0, 25.0)
ax.set_ybound(-25.0, 25.0)
ax.invert_xaxis()

# Evaluate the NCS height at each grid point.
Bz_IMF = -7.0
BZ = np.full(X.shape, Bz_IMF)
ZS = Zs_empirical(P, BY, BZ, PSI, RHO, PHI)
i = nx//2
YY = Y[..., i]
ZZ = ZS[..., i]
ax = fig.add_subplot(gs[1])
ax.grid()
ax.set_xlabel("$Y_{SM}$")
ax.set_ylabel("$Z_{SM}$")
ax.set_aspect('equal')
ax.plot(YY, ZZ, color='red')
ax.set_xbound(-25.0, 25.0)
ax.set_ybound(-25.0, 25.0)
ax.invert_xaxis()

plt.show()

In [ ]:
# Reconstruct Figure 9

# Left plot
P_dyn = 2.0
By_IMF = 0.0
psi = np.radians(30)

# Domain limits and point spacing
xmin, xmax, nx = -30.0, 10.0, 401
ymin, ymax, ny = -20.0, 20.0, 401
x = np.linspace(xmin, xmax, nx)
y = np.linspace(ymin, ymax, ny)

# Create the grids of evaluation points and variables.
X, Y = np.meshgrid(x, y)
RHO = np.sqrt(X**2 + Y**2)
PHI = np.arctan2(Y, X)
P = np.full(X.shape, P_dyn)
BY = np.full(X.shape, By_IMF)
PSI = np.full(X.shape, psi)

# Create the figure.
figsize = (10, 5)
NROWS = 1
NCOLS = 2
fig = plt.figure(figsize=figsize)
fig.suptitle('Figure 9')
gs = mpl.gridspec.GridSpec(NROWS, NCOLS)

# Evaluate the NCS height at each grid point.
Bz_IMF = 7.0
BZ = np.full(X.shape, Bz_IMF)
ZS = Zs_empirical(P, BY, BZ, PSI, RHO, PHI)
i = 0
YY = Y[..., i]
ZZ = ZS[..., i]
ax = fig.add_subplot(gs[0])
ax.grid()
ax.set_xlabel("$Y_{SM}$")
ax.set_ylabel("$Z_{SM}$")
ax.set_aspect('equal')
ax.plot(YY, ZZ, color='red')
ax.set_xbound(-25.0, 25.0)
ax.set_ybound(-25.0, 25.0)
ax.invert_xaxis()

# Evaluate the NCS height at each grid point.
Bz_IMF = -7.0
BZ = np.full(X.shape, Bz_IMF)
ZS = Zs_empirical(P, BY, BZ, PSI, RHO, PHI)
i = 0
YY = Y[..., i]
ZZ = ZS[..., i]
ax = fig.add_subplot(gs[1])
ax.grid()
ax.set_xlabel("$Y_{SM}$")
ax.set_ylabel("$Z_{SM}$")
ax.set_aspect('equal')
ax.plot(YY, ZZ, color='red')
ax.set_xbound(-25.0, 25.0)
ax.set_ybound(-25.0, 25.0)
ax.invert_xaxis()

plt.show()

In [ ]:
# Reconstruct Figure 10

# Left plot
P_dyn = 2.0
By_IMF = -7.0
psi = np.radians(0)

# Domain limits and point spacing
xmin, xmax, nx = -30.0, 10.0, 401
ymin, ymax, ny = -20.0, 20.0, 401
x = np.linspace(xmin, xmax, nx)
y = np.linspace(ymin, ymax, ny)

# Create the grids of evaluation points and variables.
X, Y = np.meshgrid(x, y)
RHO = np.sqrt(X**2 + Y**2)
PHI = np.arctan2(Y, X)
P = np.full(X.shape, P_dyn)
BY = np.full(X.shape, By_IMF)
PSI = np.full(X.shape, psi)

# Create the figure.
figsize = (10, 5)
NROWS = 1
NCOLS = 2
fig = plt.figure(figsize=figsize)
fig.suptitle('Figure 10')
gs = mpl.gridspec.GridSpec(NROWS, NCOLS)

# Evaluate the NCS height at each grid point.
Bz_IMF = 7.0
BZ = np.full(X.shape, Bz_IMF)
ZS = Zs_empirical(P, BY, BZ, PSI, RHO, PHI)
i = 0
YY = Y[..., i]
ZZ = ZS[..., i]
ax = fig.add_subplot(gs[0])
ax.grid()
ax.set_xlabel("$Y_{SM}$")
ax.set_ylabel("$Z_{SM}$")
ax.set_aspect('equal')
ax.plot(YY, ZZ, color='red')
ax.set_xbound(-25.0, 25.0)
ax.set_ybound(-25.0, 25.0)
ax.invert_xaxis()

# Evaluate the NCS height at each grid point.
Bz_IMF = -7.0
BZ = np.full(X.shape, Bz_IMF)
ZS = Zs_empirical(P, BY, BZ, PSI, RHO, PHI)
i = 0
YY = Y[..., i]
ZZ = ZS[..., i]
ax = fig.add_subplot(gs[1])
ax.grid()
ax.set_xlabel("$Y_{SM}$")
ax.set_ylabel("$Z_{SM}$")
ax.set_aspect('equal')
ax.plot(YY, ZZ, color='red')
ax.set_xbound(-25.0, 25.0)
ax.set_ybound(-25.0, 25.0)
ax.invert_xaxis()

plt.show()